# Pytorch-Lightning Trainer

By setting --trainer PL, pytorch-lightning can be used to train a knowledge graph embedding model

### Using two GPUs
python main.py --model 'DistMult' --trainer PL --num_epochs 10 --gpus 2

### Using two GPUs with DDP
python main.py --model 'DistMult' --trainer PL --num_epochs 10 --accelerator gpu


### Using two GPUs with DDP with Low-Precision
python main.py --model 'DistMult' --trainer PL --num_epochs 50 --accelerator gpu --precision 16

python main.py --model 'DistMult' --trainer PL --num_epochs 50 --accelerator gpu --precision bf16


### Using two GPUs with Model Parallel ([Deep-Speed 3](https://lightning.ai/docs/pytorch/stable/advanced/model_parallel.html#deepspeed-zero-stage-3-offload)) and Low-Precision

python main.py --path_dataset_folder 'KGs/YAGO3-10' --trainer PL --accelerator gpu --strategy deepspeed_stage_3 --precision 16